In [1]:
import numpy as np
import time
import copy
from environment import IntegratorSwitchingEnv
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split
from reward_model import LagrangeReward1
import cantera as ct
from tqdm import tqdm


In [12]:
def simulate_one_superstep_from_state(env, action, start_state, dt, super_steps, next_ref_state):
    """
    Build a fresh solver and integrate EXACTLY one super-step horizon starting from 'start_state'.
    Return (cpu_time, step_error) comparing the end state to 'next_ref_state'.
    This does NOT touch the live env state.
    """
    cfg = env.solver_configs[action]
    # fresh solver & a temporary gas bound to it
    solver, gas_tmp = env._build_fresh_solver(action, start_state.copy())

    # integrate 'super_steps' micro-steps of size dt
    t0 = time.time()
    try:
        current_state = start_state.copy()
        for _ in range(super_steps):
            if cfg['type'] == 'cvode':
                solver.set_state(current_state, 0.0)
                current_state = solver.solve_to(dt)
            elif cfg['type'] == 'qss':
                solver.setState(current_state.tolist(), 0.0)
                rc = solver.integrateToTime(dt)
                if rc != 0:
                    raise RuntimeError(f"QSS integrateToTime failed rc={rc}")
                current_state = np.array(solver.y)
            else:
                raise ValueError(f"Unknown solver type {cfg['type']}")
            # keep gas_tmp coherent (not strictly required for error)
            gas_tmp.TPY = current_state[0], env.current_pressure, current_state[1:]
        cpu_time = time.time() - t0

        # compute step error vs next reference state
        step_err = env._calculate_error(current_state, next_ref_state)
        return cpu_time, step_err

    except Exception:
        return np.inf, np.inf


def reference_anchored_oracle(env, k_step):
    """
    k_step indexes the decision point in the episode (0-based).
    We:
      - take start_ref = ref_states[k_index]
      - take next_ref  = ref_states[k_index + 1]
      where k_index = k_step * env.super_steps
    Evaluate both actions from start_ref and pick the fastest that satisfies epsilon.
    Fallback to BDF (assume action 0) otherwise.

    Returns: (best_action, (cpu_qss, err_qss), (cpu_bdf, err_bdf), obs_from_start_ref)
    """
    # map indices in the reference arrays
    start_idx = k_step * env.super_steps
    next_idx  = (k_step + 1) * env.super_steps
    if next_idx >= len(env.ref_states):
        # out of reference horizon, default to BDF
        start_idx = max(0, len(env.ref_states) - env.super_steps - 1)
        next_idx  = min(len(env.ref_states) - 1, start_idx + env.super_steps)

    start_ref = env.ref_states[start_idx]
    next_ref  = env.ref_states[next_idx]

    # Evaluate both actions from the same (reference) start state
    # Ensure action indices match your solver_configs: assume 0=BDF, 1=QSS
    cpu_bdf, err_bdf = simulate_one_superstep_from_state(env, 0, start_ref, env.dt, env.super_steps, next_ref)
    cpu_qss, err_qss = simulate_one_superstep_from_state(env, 1, start_ref, env.dt, env.super_steps, next_ref)

    eps = env.reward_function.epsilon
    # choose fastest feasible
    feasible = []
    if err_bdf <= eps: feasible.append((0, cpu_bdf))
    if err_qss <= eps: feasible.append((1, cpu_qss))

    if feasible:
        feasible.sort(key=lambda x: x[1])
        best_action = feasible[0][0]
    else:
        best_action = 0  # fallback to BDF when neither meets epsilon

    # Build features from the SAME reference state (prevents drift leakage)
    obs_ref = env._obs_from_arbitrary_state(start_ref, set_last_obs=False)
    return best_action, (cpu_qss, err_qss), (cpu_bdf, err_bdf), obs_ref


def build_ref_anchored_dataset(env_maker,
                               n_episodes=150,
                               max_steps_per_ep=200,
                               save_path="oracle_dataset_ref.npz",
                               verbose_every=10):
    """
    For each episode:
      - env.reset() to generate a fresh reference trajectory
      - at decision k, compute label by comparing solvers FROM ref[k] TO ref[k+1]
      - store obs computed from ref[k]
      - Step the LIVE env using the oracle action (so we also log realistic rollouts if desired),
        but labels/inputs are always ref-anchored and thus drift-free.
    """
    X, y = [], []

    for ep in range(n_episodes):
        env = env_maker()
        # pick conditions that likely include ignition (curriculum helps)
        obs, info = env.reset(
            temperature=np.random.uniform(*env.temp_range),
            pressure=np.random.choice(np.arange(1, 6)) * ct.one_atm,
            phi=np.random.uniform(*env.phi_range),
            total_time=np.random.uniform(*env.time_range),
            dt=np.random.uniform(*env.dt_range),
            etol=env.etol
        )

        T_ref_init = env.ref_states[0][0]
        T_ref_final = env.ref_states[-1][0]
        pressure = env.current_pressure
        phi = env.current_phi
        total_time = env.total_time
        dt = env.dt
        print(f"[episode {ep}] - T_ref_init: {T_ref_init}, T_ref_final: {T_ref_final}, pressure: {pressure}, phi: {phi}, total_time: {total_time}, dt: {dt}")
        # number of decision points = env.n_episodes
        K = env.n_episodes
        pbar = tqdm(range(min(K, max_steps_per_ep)), desc="Building reference-anchored dataset")
        for k in pbar:
            a_star, (cpu_qss, err_qss), (cpu_bdf, err_bdf), obs_ref = reference_anchored_oracle(env, k)
            X.append(obs_ref.astype(np.float32))
            y.append(a_star)

            # Advance the REAL env with the oracle action (optional but keeps trajectories sensible)
            obs, reward, terminated, truncated, info = env.step(a_star)
            if terminated or truncated:
                break
            pbar.set_postfix({
                "episode": ep,
                "step": k,
                "err_qss": err_qss,
                "err_bdf": err_bdf,
                "cpu_qss": cpu_qss,
                "cpu_bdf": cpu_bdf,
                "Ti": T_ref_init,
                "Tf": T_ref_final,
            })
            pbar.update(1)
            

            
        if (ep + 1) % verbose_every == 0:
            print(f"[ref-dataset] ep {ep+1}/{n_episodes}, total samples={len(X)}")

    X = np.stack(X, axis=0).astype(np.float32)
    y = np.array(y, dtype=np.int64)
    np.savez_compressed(save_path, X=X, y=y)
    print(f"[ref-dataset] saved {save_path} with X={X.shape}, y={y.shape}")


In [13]:



# ---- Configure solver order: 0=BDF, 1=QSS (must match oracle_refanchored.py assumptions) ----
SOLVER_CONFIGS = [
    # 0 = BDF (CVODE)
    {'type': 'cvode', 'rtol': 1e-6, 'atol': 1e-12, 'mxsteps': 100000, 'name': 'CVODE_BDF'},
    # 1 = QSS
    {'type': 'qss', 'dtmin': 1e-16, 'dtmax': 1e-6, 'stabilityCheck': False, 'itermax': 2,
     'epsmin': 0.002, 'epsmax': 100.0, 'abstol': 1e-8, 'mxsteps': 1000, 'name': 'QSS'},
]


def env_maker():
    """
    Returns a fresh IntegratorSwitchingEnv instance each call.
    The builder will call env.reset(...) with random ICs drawn from these ranges.
    Bias ranges so many episodes include ignition within the horizon.
    """
    reward_cfg = dict(
        epsilon=1e-4,
        lambda_init=1.0,
        lambda_lr=0.05,
        target_violation=0.0,
        cpu_log_delta=1e-3,
        reward_clip=5.0,
    )

    env = IntegratorSwitchingEnv(
        mechanism_file="large_mechanism/n-dodecane.yaml",  # or "gri30.yaml"
        fuel="nc12h26",                                     # or "CH4:1.0" for GRI-30
        oxidizer="O2:0.21, N2:0.79",                        # or "N2:3.76, O2:1.0"
        # Ranges the builder will sample from at reset():
        temp_range=(600.0, 1400.0),     # wide enough to see pre- & post-ignition
        phi_range=(0.7, 1.6),
        pressure_range=(1, 6),          # (ignored internally; builder passes explicit P in atm)
        time_range=(1e-3, 1e-2),        # ensure horizon can include ignition
        dt_range=(1e-6, 1e-6),          # fixed dt for comparability
        etol=1e-4,
        verbose=False,
        termination_count_threshold=100,
        reward_function=LagrangeReward1(**reward_cfg),
    )

    # Ensure solver indices: 0=BDF, 1=QSS (oracle assumes this order)
    env.solver_configs = SOLVER_CONFIGS
    return env





In [14]:
build_ref_anchored_dataset(
    env_maker=env_maker,
    n_episodes=200,            # total episodes to sample for the dataset
    max_steps_per_ep=200,      # cap decisions per episode
    save_path="oracle_dataset_ref.npz",
    verbose_every=10
)

print("Saved dataset to oracle_dataset_ref.npz")


[episode 0] - T_ref_init: 718.0930049591914, T_ref_final: 957.6584620278323, pressure: 405300.0, phi: 0.9080777256685133, total_time: 0.007782272165221903, dt: 1e-06


Building reference-anchored dataset:  99%|█████████▉| 100/101 [00:56<00:00,  1.78it/s, episode=0, step=99, err_qss=5.78e-6, err_bdf=1.64e-6, cpu_qss=0.00779, cpu_bdf=0.183, Ti=718, Tf=958] 


[episode 1] - T_ref_init: 1065.1312190553303, T_ref_final: 1046.6313151902064, pressure: 303975.0, phi: 1.5548787318305848, total_time: 0.002183875543322598, dt: 1e-06


Building reference-anchored dataset:  99%|█████████▉| 102/103 [00:33<00:00,  3.00it/s, episode=1, step=101, err_qss=8.65e-6, err_bdf=6.55e-6, cpu_qss=0.00217, cpu_bdf=0.0511, Ti=1.07e+3, Tf=1.05e+3]


[episode 2] - T_ref_init: 776.1164037230996, T_ref_final: 956.5259271747864, pressure: 506625.0, phi: 1.0231915022017108, total_time: 0.003583343999156343, dt: 1e-06


...

[episode 129] - T_ref_init: 837.935201350857, T_ref_final: 865.2893553295759, pressure: 303975.0, phi: 1.215082358569828, total_time: 0.005675146259033287, dt: 1e-06


Building reference-anchored dataset:  99%|█████████▉| 100/101 [01:47<00:01,  1.07s/it, episode=129, step=99, err_qss=5.66e-6, err_bdf=4.49e-7, cpu_qss=0.00561, cpu_bdf=0.135, Ti=838, Tf=865]


[ref-dataset] ep 130/200, total samples=13121
[episode 130] - T_ref_init: 829.8376873177758, T_ref_final: 888.93690182347, pressure: 405300.0, phi: 0.8677747866762152, total_time: 0.0037153415498548067, dt: 1e-06


Building reference-anchored dataset:   0%|          | 0/100 [00:00<?, ?it/s]

: 

In [ ]:

class SwitcherMLP(nn.Module):
    def __init__(self, in_dim, hidden=128, dropout=0.1):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(in_dim, hidden),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden, hidden),
            nn.ReLU(),
            nn.Linear(hidden, 2)  # 0=BDF, 1=QSS
        )

    def forward(self, x):
        return self.net(x)

def train_classifier(dataset_path="oracle_dataset.npz",
                     epochs=15,
                     batch_size=512,
                     lr=3e-4,
                     out_path="switcher_classifier.pt"):
    data = np.load(dataset_path)
    X = data["X"].astype(np.float32)
    y = data["y"].astype(np.int64)

    Xtr, Xval, ytr, yval = train_test_split(X, y, test_size=0.15, random_state=42, stratify=y)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = SwitcherMLP(in_dim=X.shape[1]).to(device)

    train_ds = TensorDataset(torch.from_numpy(Xtr), torch.from_numpy(ytr))
    val_ds   = TensorDataset(torch.from_numpy(Xval), torch.from_numpy(yval))
    train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True, drop_last=False)
    val_loader   = DataLoader(val_ds, batch_size=2048, shuffle=False, drop_last=False)

    opt = torch.optim.AdamW(model.parameters(), lr=lr)
    crit = nn.CrossEntropyLoss()

    best_acc = 0.0
    for ep in range(1, epochs+1):
        model.train()
        tr_loss, tr_n = 0.0, 0
        for xb, yb in train_loader:
            xb, yb = xb.to(device), yb.to(device)
            opt.zero_grad()
            logits = model(xb)
            loss = crit(logits, yb)
            loss.backward()
            nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            opt.step()
            tr_loss += loss.item() * xb.size(0)
            tr_n += xb.size(0)
        tr_loss /= tr_n

        # val
        model.eval()
        correct, tot = 0, 0
        with torch.no_grad():
            for xb, yb in val_loader:
                xb, yb = xb.to(device), yb.to(device)
                logits = model(xb)
                pred = logits.argmax(dim=1)
                correct += (pred == yb).sum().item()
                tot += xb.size(0)
        acc = correct / tot
        print(f"[clf] ep {ep:02d} | train_loss {tr_loss:.4f} | val_acc {acc:.4f}")

        if acc > best_acc:
            best_acc = acc
            torch.save(model.state_dict(), out_path)
            print(f"[clf] saved best to {out_path} (acc={best_acc:.4f})")

    print(f"[clf] done. best val acc={best_acc:.4f}")

if __name__ == "__main__":
    train_classifier()
